# GAL_SIM 市場規模推定・セグメント分析

## 目的
- 日本のアダルトコンテンツ市場データから統計分析
- セグメント規模、課金率、ARPU、LTVを推定
- 最大収益セグメントの特定
- 初年度収益予測（3シナリオ）

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:,.0f}'.format)

## 1. 市場規模推定（TAM/SAM/SOM）

In [ ]:
# ========================================
# 基礎データ: 日本の人口統計
# ========================================

# 日本18-45歳男性人口（2024年推定）
total_male_18_45 = 18_500_000  # 約1850万人

# 年齢層別内訳
male_18_24 = 3_800_000   # 18-24歳（学生中心）
male_25_34 = 6_200_000   # 25-34歳（若手社会人）
male_35_45 = 8_500_000   # 35-45歳（中堅社会人）

# ========================================
# 基礎データ: アダルトコンテンツ市場
# ========================================

# FANZA市場データ（2021年）
fanza_revenue_2021 = 105_000_000_000  # 1,050億円
fanza_market_share = 0.70  # 市場シェア70%

# 総市場規模推定
total_adult_market = fanza_revenue_2021 / fanza_market_share  # 約1,500億円

# DLsite同人市場の成長率（年平均）
dlsite_growth_rate = 0.15  # 15%成長

# アダルトコンテンツ消費者率（18-45歳男性）
adult_content_consumer_rate = 0.45  # 45%（推定）
adult_content_consumers = int(total_male_18_45 * adult_content_consumer_rate)

# ギャルゲー・ノベルゲーム層の比率（アダルトコンテンツ消費者の中）
galge_rate = 0.12  # 12%（推定）
galge_consumers = int(adult_content_consumers * galge_rate)

print("="*60)
print("市場規模推定: 基礎データ")
print("="*60)
print(f"日本18-45歳男性人口: {total_male_18_45:,}人")
print(f"  - 18-24歳（学生中心）: {male_18_24:,}人")
print(f"  - 25-34歳（若手社会人）: {male_25_34:,}人")
print(f"  - 35-45歳（中堅社会人）: {male_35_45:,}人")
print()
print(f"アダルトコンテンツ市場規模: ¥{total_adult_market:,.0f}")
print(f"アダルトコンテンツ消費者: {adult_content_consumers:,}人 ({adult_content_consumer_rate:.0%})")
print(f"ギャルゲー・ノベルゲーム層: {galge_consumers:,}人 ({galge_rate:.0%})")
print()

In [ ]:
# ========================================
# AI使用者層の推定
# ========================================

# ChatGPT日本ユーザー数（2024年推定）
chatgpt_japan_users = 5_000_000  # 約500万人

# 18-45歳男性の中でのAI使用者率（推定）
ai_user_rate = 0.18  # 18%（若年層で高い）

# 年齢層別AI使用率（推定）
ai_rate_18_24 = 0.25  # 25%（学生・若年層で高い）
ai_rate_25_34 = 0.20  # 20%（IT系社会人で高い）
ai_rate_35_45 = 0.12  # 12%（中堅層で低め）

# ギャルゲー層の中でのAI使用率（全体より高いと仮定）
ai_rate_galge = 0.22  # 22%（テック親和性が高い層）

galge_ai_users = int(galge_consumers * ai_rate_galge)
galge_non_ai_users = galge_consumers - galge_ai_users

print("="*60)
print("AI使用者層の推定")
print("="*60)
print(f"ChatGPT日本ユーザー数: {chatgpt_japan_users:,}人")
print(f"18-45歳男性のAI使用率: {ai_user_rate:.0%}")
print()
print(f"ギャルゲー層のAI使用率: {ai_rate_galge:.0%}")
print(f"  - AI使用者: {galge_ai_users:,}人")
print(f"  - AI非使用者: {galge_non_ai_users:,}人")
print()

In [ ]:
# ========================================
# TAM/SAM/SOM推定
# ========================================

# TAM（Total Addressable Market）
tam = galge_consumers

# SAM（Serviceable Available Market）- 初年度リーチ可能
# 広告予算とマーケティング効率を考慮
initial_ad_budget = 500_000  # 初期広告予算: 50万円
cpm = 1_000  # CPM（1,000インプレッションあたり）
ctr = 0.02  # クリック率: 2%
conversion_to_download = 0.30  # DL率: 30%

# リーチ可能人数
impressions = (initial_ad_budget / cpm) * 1_000
clicks = int(impressions * ctr)
sam = int(clicks * conversion_to_download)

# SOM（Serviceable Obtainable Market）- 初年度実DL数（保守的）
som_conservative = int(sam * 0.50)  # 保守シナリオ: 50%
som_base = sam  # 基本シナリオ: 100%
som_optimistic = int(sam * 2.0)  # 楽観シナリオ: バイラル効果2倍

print("="*60)
print("TAM / SAM / SOM 推定")
print("="*60)
print(f"TAM（Total Addressable Market）: {tam:,}人")
print(f"  - ギャルゲー・ノベルゲーム消費者全体")
print()
print(f"SAM（Serviceable Available Market）: {sam:,}人")
print(f"  - 初年度リーチ可能（広告予算¥{initial_ad_budget:,}想定）")
print(f"  - インプレッション: {impressions:,.0f}")
print(f"  - クリック数: {clicks:,}（CTR {ctr:.0%}）")
print(f"  - DL率: {conversion_to_download:.0%}")
print()
print(f"SOM（Serviceable Obtainable Market）:")
print(f"  - 保守シナリオ: {som_conservative:,}人")
print(f"  - 基本シナリオ: {som_base:,}人")
print(f"  - 楽観シナリオ: {som_optimistic:,}人")
print()

## 2. セグメント規模の推定

In [ ]:
# ========================================
# セグメント定義
# ========================================

segments = {
    '学生（AI使用）': {
        'age_group': '18-24',
        'ai_user': True,
        'population_base': male_18_24,
        'galge_rate': 0.15,  # 学生はギャルゲー率高め
        'ai_rate': ai_rate_18_24,
    },
    '学生（非AI）': {
        'age_group': '18-24',
        'ai_user': False,
        'population_base': male_18_24,
        'galge_rate': 0.15,
        'ai_rate': 1 - ai_rate_18_24,
    },
    '若手社会人（AI使用）': {
        'age_group': '25-34',
        'ai_user': True,
        'population_base': male_25_34,
        'galge_rate': 0.12,
        'ai_rate': ai_rate_25_34,
    },
    '若手社会人（非AI）': {
        'age_group': '25-34',
        'ai_user': False,
        'population_base': male_25_34,
        'galge_rate': 0.12,
        'ai_rate': 1 - ai_rate_25_34,
    },
    '中堅社会人（AI使用）': {
        'age_group': '35-45',
        'ai_user': True,
        'population_base': male_35_45,
        'galge_rate': 0.10,  # 中堅はギャルゲー率低め
        'ai_rate': ai_rate_35_45,
    },
    '中堅社会人（非AI）': {
        'age_group': '35-45',
        'ai_user': False,
        'population_base': male_35_45,
        'galge_rate': 0.10,
        'ai_rate': 1 - ai_rate_35_45,
    },
}

# セグメント別TAM計算
for segment_name, data in segments.items():
    data['tam'] = int(
        data['population_base'] * 
        adult_content_consumer_rate * 
        data['galge_rate'] * 
        data['ai_rate']
    )

# TAM合計検証
total_tam = sum(data['tam'] for data in segments.values())

print("="*60)
print("セグメント別TAM推定")
print("="*60)
for segment_name, data in segments.items():
    print(f"{segment_name:20s}: {data['tam']:>8,}人")
print("-"*60)
print(f"{'合計':20s}: {total_tam:>8,}人")
print(f"{'検証（全体TAM）':20s}: {tam:>8,}人")
print()

## 3. 課金率・ARPU・LTVの推定

In [ ]:
# ========================================
# ベンチマークデータ
# ========================================

# DLsite同人ゲームの平均データ
dlsite_avg_price = 800  # 平均価格: 800円
dlsite_conversion_rate = 0.015  # 訪問→購入率: 1.5%

# FANZAの平均購入額（動画・ゲーム含む）
fanza_avg_purchase = 1_200  # 平均購入額: 1,200円
fanza_monthly_user = 3_000_000  # 月間アクティブユーザー

# ソシャゲの課金率・ARPU（参考）
mobile_game_conversion = 0.03  # 課金率: 3%
mobile_game_arpu = 1_500  # ARPU: 1,500円/月

# GAL_SIMの価格設定
galsim_free_price = 0  # 無料版（STAGE 3まで）
galsim_premium_price = 500  # プレミアム版: 500円（STAGE 4解放）

print("="*60)
print("ベンチマークデータ")
print("="*60)
print(f"DLsite同人ゲーム平均価格: ¥{dlsite_avg_price:,}")
print(f"DLsite訪問→購入率: {dlsite_conversion_rate:.1%}")
print()
print(f"FANZA平均購入額: ¥{fanza_avg_purchase:,}")
print(f"FANZA月間アクティブユーザー: {fanza_monthly_user:,}人")
print()
print(f"ソシャゲ課金率: {mobile_game_conversion:.1%}")
print(f"ソシャゲARPU: ¥{mobile_game_arpu:,}/月")
print()
print(f"GAL_SIMプレミアム価格: ¥{galsim_premium_price:,}")
print()

In [ ]:
# ========================================
# セグメント別 課金率・ARPU・LTV推定
# ========================================

# セグメント別の特性定義
segment_characteristics = {
    '学生（AI使用）': {
        'conversion_rate': 0.08,  # 8%（AI親和性高いが可処分所得低い）
        'repeat_play_rate': 2.5,  # 平均2.5回プレイ
        'word_of_mouth': 1.3,  # 口コミ効果1.3倍
    },
    '学生（非AI）': {
        'conversion_rate': 0.05,  # 5%（可処分所得低い）
        'repeat_play_rate': 2.0,
        'word_of_mouth': 1.1,
    },
    '若手社会人（AI使用）': {
        'conversion_rate': 0.15,  # 15%（可処分所得あり、AI親和性高い）
        'repeat_play_rate': 3.0,
        'word_of_mouth': 1.2,
    },
    '若手社会人（非AI）': {
        'conversion_rate': 0.12,  # 12%（可処分所得あり）
        'repeat_play_rate': 2.8,
        'word_of_mouth': 1.0,
    },
    '中堅社会人（AI使用）': {
        'conversion_rate': 0.18,  # 18%（可処分所得高い、AI親和性高い）
        'repeat_play_rate': 2.5,
        'word_of_mouth': 1.0,
    },
    '中堅社会人（非AI）': {
        'conversion_rate': 0.10,  # 10%（可処分所得高いがAI親和性低い）
        'repeat_play_rate': 2.2,
        'word_of_mouth': 0.9,
    },
}

# データをセグメントに統合
for segment_name, char in segment_characteristics.items():
    segments[segment_name].update(char)
    
    # ARPU計算（1回のプレイあたりの期待収益）
    segments[segment_name]['arpu'] = int(
        galsim_premium_price * char['conversion_rate']
    )
    
    # LTV計算（リピート率を考慮）
    segments[segment_name]['ltv'] = int(
        segments[segment_name]['arpu'] * char['repeat_play_rate']
    )

# セグメント別データをDataFrameに整理
segment_df = pd.DataFrame([
    {
        'セグメント': name,
        'TAM': data['tam'],
        '課金率': data['conversion_rate'],
        'ARPU': data['arpu'],
        'リピート率': data['repeat_play_rate'],
        'LTV': data['ltv'],
        '年間収益ポテンシャル': int(data['tam'] * data['ltv']),
    }
    for name, data in segments.items()
])

# 収益ポテンシャルでソート
segment_df = segment_df.sort_values('年間収益ポテンシャル', ascending=False)

print("="*80)
print("セグメント別 課金率・ARPU・LTV推定")
print("="*80)
print(segment_df.to_string(index=False))
print()
print(f"合計年間収益ポテンシャル: ¥{segment_df['年間収益ポテンシャル'].sum():,}")
print()

## 4. 最大収益セグメントの特定

In [ ]:
# ========================================
# 最大収益セグメント分析
# ========================================

top_segment = segment_df.iloc[0]

print("="*60)
print("最大収益セグメント")
print("="*60)
print(f"セグメント名: {top_segment['セグメント']}")
print()
print(f"TAM（市場規模）: {top_segment['TAM']:,}人")
print(f"課金率: {top_segment['課金率']:.1%}")
print(f"ARPU（1回あたり）: ¥{top_segment['ARPU']:,}")
print(f"リピート率: {top_segment['リピート率']:.1f}回")
print(f"LTV（生涯価値）: ¥{top_segment['LTV']:,}")
print(f"年間収益ポテンシャル: ¥{top_segment['年間収益ポテンシャル']:,}")
print()
print("理由:")
print(f"1. 規模: {top_segment['TAM']:,}人（全体の{top_segment['TAM']/total_tam:.1%}）")
print(f"2. 課金率: {top_segment['課金率']:.1%}（高い）")
print(f"3. LTV: ¥{top_segment['LTV']:,}（高い）")
print(f"4. 年間収益: ¥{top_segment['年間収益ポテンシャル']:,}（最大）")
print()

In [ ]:
# ========================================
# セグメント別収益ポテンシャルの可視化
# ========================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. TAM比較
ax1 = axes[0, 0]
segment_df.plot.barh(x='セグメント', y='TAM', ax=ax1, color='steelblue', legend=False)
ax1.set_title('セグメント別TAM（市場規模）', fontsize=14, fontweight='bold')
ax1.set_xlabel('人数', fontsize=12)
ax1.set_ylabel('')

# 2. 課金率比較
ax2 = axes[0, 1]
segment_df.plot.barh(x='セグメント', y='課金率', ax=ax2, color='coral', legend=False)
ax2.set_title('セグメント別課金率', fontsize=14, fontweight='bold')
ax2.set_xlabel('課金率', fontsize=12)
ax2.set_ylabel('')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))

# 3. LTV比較
ax3 = axes[1, 0]
segment_df.plot.barh(x='セグメント', y='LTV', ax=ax3, color='seagreen', legend=False)
ax3.set_title('セグメント別LTV（生涯価値）', fontsize=14, fontweight='bold')
ax3.set_xlabel('LTV（円）', fontsize=12)
ax3.set_ylabel('')

# 4. 年間収益ポテンシャル比較
ax4 = axes[1, 1]
segment_df.plot.barh(x='セグメント', y='年間収益ポテンシャル', ax=ax4, color='crimson', legend=False)
ax4.set_title('セグメント別年間収益ポテンシャル', fontsize=14, fontweight='bold')
ax4.set_xlabel('収益（円）', fontsize=12)
ax4.set_ylabel('')

plt.tight_layout()
plt.savefig('/Users/kenjihachiya/Desktop/work/development_gemini/av/galg/gal-sim/gal-sim/segment_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("グラフを保存しました: segment_analysis.png")

## 5. 初年度収益予測（3シナリオ）

In [ ]:
# ========================================
# 初年度収益予測: 保守・基本・楽観シナリオ
# ========================================

# 各セグメントのDL数分布を推定
# （マーケティング施策によってセグメント別にリーチ率が異なる）

segment_reach_rate = {
    '学生（AI使用）': 0.15,
    '学生（非AI）': 0.08,
    '若手社会人（AI使用）': 0.20,
    '若手社会人（非AI）': 0.12,
    '中堅社会人（AI使用）': 0.18,
    '中堅社会人（非AI）': 0.10,
}

scenarios = {
    '保守': {
        'total_downloads': som_conservative,
        'multiplier': 1.0,
    },
    '基本': {
        'total_downloads': som_base,
        'multiplier': 1.0,
    },
    '楽観': {
        'total_downloads': som_optimistic,
        'multiplier': 1.0,
    },
}

# シナリオ別の収益計算
results = []

for scenario_name, scenario_data in scenarios.items():
    total_downloads = scenario_data['total_downloads']
    
    # セグメント別DL数推定
    segment_downloads = {}
    for segment_name, reach_rate in segment_reach_rate.items():
        segment_downloads[segment_name] = int(total_downloads * reach_rate)
    
    # 課金収益計算
    total_revenue = 0
    total_paying_users = 0
    
    for segment_name, downloads in segment_downloads.items():
        conversion_rate = segments[segment_name]['conversion_rate']
        paying_users = int(downloads * conversion_rate)
        revenue = paying_users * galsim_premium_price
        
        total_revenue += revenue
        total_paying_users += paying_users
    
    # 平均課金率
    avg_conversion_rate = total_paying_users / total_downloads if total_downloads > 0 else 0
    
    results.append({
        'シナリオ': scenario_name,
        'DL数': total_downloads,
        '課金ユーザー数': total_paying_users,
        '平均課金率': avg_conversion_rate,
        '収益': total_revenue,
    })

results_df = pd.DataFrame(results)

print("="*80)
print("初年度収益予測（3シナリオ）")
print("="*80)
print(results_df.to_string(index=False))
print()

# シナリオ別詳細
for i, row in results_df.iterrows():
    print(f"## {row['シナリオ']}シナリオ")
    print(f"- DL数: {row['DL数']:,}人")
    print(f"- 課金ユーザー数: {row['課金ユーザー数']:,}人")
    print(f"- 平均課金率: {row['平均課金率']:.1%}")
    print(f"- 収益: ¥{row['収益']:,}")
    print()

In [ ]:
# ========================================
# 収益予測の可視化
# ========================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. DL数比較
ax1 = axes[0]
results_df.plot.bar(x='シナリオ', y='DL数', ax=ax1, color='steelblue', legend=False)
ax1.set_title('シナリオ別DL数予測', fontsize=14, fontweight='bold')
ax1.set_xlabel('シナリオ', fontsize=12)
ax1.set_ylabel('DL数（人）', fontsize=12)
ax1.tick_params(axis='x', rotation=0)

# 2. 収益比較
ax2 = axes[1]
results_df.plot.bar(x='シナリオ', y='収益', ax=ax2, color='crimson', legend=False)
ax2.set_title('シナリオ別収益予測', fontsize=14, fontweight='bold')
ax2.set_xlabel('シナリオ', fontsize=12)
ax2.set_ylabel('収益（円）', fontsize=12)
ax2.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('/Users/kenjihachiya/Desktop/work/development_gemini/av/galg/gal-sim/gal-sim/revenue_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

print("グラフを保存しました: revenue_forecast.png")

## 6. 総括レポート出力

In [ ]:
# ========================================
# 総括レポート生成
# ========================================

report = f"""
{'='*80}
GAL_SIM 市場規模推定・セグメント分析 総括レポート
{'='*80}
作成日時: {datetime.now().strftime('%Y年%m月%d日 %H:%M')}

# 市場規模推定

## TAM（Total Addressable Market）
- 日本18-45歳男性: {total_male_18_45:,}人
- アダルトコンテンツ消費者: {adult_content_consumers:,}人（{adult_content_consumer_rate:.0%}）
- ギャルゲー・ノベルゲーム層: {galge_consumers:,}人（{galge_rate:.0%}）
- GAL_SIM TAM: {tam:,}人

## SAM（Serviceable Available Market）
- 初年度リーチ可能: {sam:,}人（広告予算¥{initial_ad_budget:,}想定）

## SOM（Serviceable Obtainable Market）
- 保守シナリオ: {som_conservative:,}人
- 基本シナリオ: {som_base:,}人
- 楽観シナリオ: {som_optimistic:,}人

{'='*80}

# セグメント規模推定

{segment_df.to_string(index=False)}

{'='*80}

# 最大収益セグメント

**答え: {top_segment['セグメント']}**

理由:
1. 規模: {top_segment['TAM']:,}人（全体の{top_segment['TAM']/total_tam:.1%}）
2. 課金率: {top_segment['課金率']:.1%}（高い）
3. ARPU: ¥{top_segment['ARPU']:,}（高い）
4. LTV: ¥{top_segment['LTV']:,}（高い）
5. 年間収益: ¥{top_segment['年間収益ポテンシャル']:,}（最大）

{'='*80}

# 初年度収益予測（3シナリオ）

## 保守シナリオ
- DL数: {results_df.iloc[0]['DL数']:,}人
- 課金ユーザー数: {results_df.iloc[0]['課金ユーザー数']:,}人
- 平均課金率: {results_df.iloc[0]['平均課金率']:.1%}
- 収益: ¥{results_df.iloc[0]['収益']:,}

## 基本シナリオ
- DL数: {results_df.iloc[1]['DL数']:,}人
- 課金ユーザー数: {results_df.iloc[1]['課金ユーザー数']:,}人
- 平均課金率: {results_df.iloc[1]['平均課金率']:.1%}
- 収益: ¥{results_df.iloc[1]['収益']:,}

## 楽観シナリオ
- DL数: {results_df.iloc[2]['DL数']:,}人
- 課金ユーザー数: {results_df.iloc[2]['課金ユーザー数']:,}人
- 平均課金率: {results_df.iloc[2]['平均課金率']:.1%}
- 収益: ¥{results_df.iloc[2]['収益']:,}

{'='*80}

# 推奨マーケティング戦略

## 最優先ターゲット: {top_segment['セグメント']}

### 理由
1. 最大の収益ポテンシャル（¥{top_segment['年間収益ポテンシャル']:,}）
2. 高い課金率（{top_segment['課金率']:.1%}）
3. 高いLTV（¥{top_segment['LTV']:,}）

### マーケティング施策
1. **AI親和性の高いチャネル**: ChatGPT/Gemini関連コミュニティでの展開
2. **可処分所得を考慮した訴求**: 「15分で完結」「サクッとプレイ」
3. **課金壁の最適化**: STAGE 3での焦らしを最大化

## サブターゲット
1. 若手社会人（非AI）: 規模大、課金率良好
2. 中堅社会人（AI使用）: 課金率最高、LTV高い

{'='*80}

# データの前提と制約

## データソース
- 日本の人口統計: 総務省統計データ（2024年推定）
- FANZA市場データ: 公開情報（2021年）
- DLsite同人市場: 業界レポート
- ChatGPTユーザー数: 推定値

## 推定の前提
- アダルトコンテンツ消費者率: 45%（推定）
- ギャルゲー層比率: 12%（推定）
- AI使用率: 年齢層別に18-25%（推定）
- 課金率: セグメント別に5-18%（ベンチマーク＋推定）

## 制約
- 実際の課金率はコンテンツ品質に大きく依存
- バイラル効果は予測困難
- 競合の動向は考慮していない

{'='*80}
"""

print(report)

# レポートをファイルに保存
with open('/Users/kenjihachiya/Desktop/work/development_gemini/av/galg/gal-sim/gal-sim/market_analysis_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\nレポートを保存しました: market_analysis_report.txt")

In [ ]:
# ========================================
# セグメント別データをCSVエクスポート
# ========================================

segment_df.to_csv(
    '/Users/kenjihachiya/Desktop/work/development_gemini/av/galg/gal-sim/gal-sim/segment_data.csv',
    index=False,
    encoding='utf-8-sig'
)

results_df.to_csv(
    '/Users/kenjihachiya/Desktop/work/development_gemini/av/galg/gal-sim/gal-sim/revenue_forecast.csv',
    index=False,
    encoding='utf-8-sig'
)

print("データをCSVファイルにエクスポートしました:")
print("- segment_data.csv")
print("- revenue_forecast.csv")